# 5장 선형 회귀

* "부록3 매트플롯립 입문"에서 한글 폰트를 올바르게 출력하기 위한 설치 방법을 설명했다. 설치 방법은 다음과 같다.

In [ ]:
!sudo apt-get install -y fonts-nanum* | tail -n 1
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

In [ ]:
# 필요 라이브러리 설치

!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

* 모든 설치가 끝나면 한글 폰트를 바르게 출력하기 위해 **[런타임]** -> **[런타임 다시시작]**을 클릭한 다음, 아래 셀부터 코드를 실행해 주십시오.

In [ ]:
# 라이브러리 임포트

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 폰트 관련 용도
import matplotlib.font_manager as fm

# 나눔 고딕 폰트의 경로 명시
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

In [ ]:
# 기본 폰트 설정
plt.rcParams['font.family'] = font_name

# 기본 폰트 사이즈 변경
plt.rcParams['font.size'] = 14

# 기본 그래프 사이즈 변경
plt.rcParams['figure.figsize'] = (6,6)

# 기본 그리드 표시
# 필요에 따라 설정할 때는, plt.grid()
plt.rcParams['axes.grid'] = True

# 마이너스 기호 정상 출력
plt.rcParams['axes.unicode_minus'] = False

# 넘파이 부동소수점 자릿수 표시
np.set_printoptions(suppress=True, precision=4)

## 5.3 선형 함수(nn.Linear)

###  입력 :1 출력 :1인 선형 함수

In [ ]:
# 코드 5-1: 선형 함수의 인스턴스 생성
# 입력이 2차원, 출력이 3차원인 선형 함수를 정의합니다.
l = nn.Linear(2, 3)
print(l)
# w1x1 + w2x2 + b

In [ ]:
# 코드 5-2 & 5-3: 1입력 1출력 선형 함수 생성 및 내부 파라미터 확인

# 난수 시드값 고정 (항상 동일한 결과를 얻기 위함)
torch.manual_seed(123)

# 입력: 1, 출력: 1 선형 함수를 정의합니다.
# y = w1x1 + b
l1 = nn.Linear(1, 1)

# 생성된 선형 함수를 확인합니다.
print("l1 정보:")
print(l1)

In [ ]:
# l1 내부의 파라미터(weight, bias)를 확인합니다.
print("\nl1의 파라미터:")
for param in l1.named_parameters():
    print('name  :', param[0])       # 파라미터 이름 출력
    print('tensor:', param[1])       # 파라미터 텐서 값 출력
    print('shape :', param[1].shape) # 파라미터 텐서의 모양 출력

In [ ]:
# 코드 5-4: 레이어 함수 안의 파라미터 초깃값 설정

# 초깃값 설정 (weight=2.0, bias=1.0)
nn.init.constant_(l1.weight, 2.0) # weight 텐서의 모든 값을 2.0으로 설정
nn.init.constant_(l1.bias, 1.0)   # bias 텐서의 모든 값을 1.0으로 설정

# 결과 확인
print("weight:", l1.weight)
print("bias:", l1.bias)

In [ ]:
np.arange(-2.0, 2.1, 1.0)

In [ ]:
# 코드 5-5: 테스트용 변수 준비

# -2부터 2까지 1씩 증가하는 NumPy 배열 생성
x_np = np.arange(-2.0, 2.1, 1.0)
print(x_np)
# NumPy 배열을 PyTorch 텐서로 변환
x = torch.tensor(x_np).float() # .float()으로 데이터 타입을 float32로 지정
print(x)

# (N, 1) 사이즈로 변경. N은 데이터 개수.
# 선형 함수는 기본적으로 배치(묶음) 데이터를 처리하므로 2차원 텐서 입력을 가정합니다.
x = x.view(-1, 1)  # (5,1)
print(x)

# 결과 확인
print("x의 shape:", x.shape)
print("x의 값:\n", x)

In [ ]:
# 코드 5-6: 1차 함수 테스트

# l1 함수에 x를 입력하여 y를 계산합니다.
y = l1(x)
print(y)
# 결과 확인
print("y의 shape:", y.shape)
print("y의 data:\n", y.data) # y.data는 requires_grad 정보 없이 순수 데이터만 보여줍니다.

### 입력 :2 출력 :1인 선형 함수

In [ ]:
# 코드 5-7: 2입력 1출력 선형 함수 정의 및 초기화

# 입력: 2, 출력: 1 선형 함수 정의
l2 = nn.Linear(2, 1)

# 초깃값 설정 (모든 가중치는 1.0, 편향은 2.0으로 설정)
nn.init.constant_(l2.weight, 1.0)
nn.init.constant_(l2.bias, 2.0)

# y = w1x1 + w2x2 + b >> y = 1x1 + 1x2 + 2
# 결과 확인
print("weight:", l2.weight)
print("bias:", l2.bias)

In [ ]:
# 코드 5-8 & 5-9: 2입력 1출력 함수 테스트

# 테스트용 2차원 데이터 준비
x2_np = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
x2 = torch.tensor(x2_np).float()

print("x2의 shape:", x2.shape)
print("x2의 값:\n", x2)

In [ ]:
# 함수 값 계산
y2 = l2(x2)

# shape 확인
print(y2.shape)

# 값 확인
print(y2.data)

### 입력 :2 출력:3인 선형 함수

In [ ]:
# 코드 5-10: 2입력 3출력 선형 함수 정의 및 초기화

# 입력: 2, 출력: 3 선형 함수 정의
l3 = nn.Linear(2, 3)

# 초깃값 설정 (weight 행렬의 각 행을 다른 값으로 설정)
nn.init.constant_(l3.weight[0, :], 1.0)
nn.init.constant_(l3.weight[1, :], 2.0)
nn.init.constant_(l3.weight[2, :], 3.0)
nn.init.constant_(l3.bias, 2.0)

# 결과 확인
print("weight:", l3.weight)
print("bias:", l3.bias)

In [ ]:
# 코드 5-11: 2입력 3출력 함수 테스트

# 함수 값 계산 (입력은 x2를 재사용)
y3 = l3(x2)

# 결과 확인
print("y3의 shape:", y3.shape)
print("y3의 data:\n", y3.data)

## 5.4 커스텀 클래스를 이용한 모델 정의

In [ ]:
# 코드 5-12 모델을 정의하는 본질적인 부분
class Net(nn.Module): # 모든 모델의 부모 클래스인 nn.Module을 상속받습니다.
    def __init__(self, n_input, n_output):
        # 부모 클래스 nn.Module의 초기화 메서드를 반드시 호출해야 합니다.
        super().__init__()

        # 출력층 정의: 입력 특성 수(n_input)와 출력 특성 수(n_output)를 받는 선형 레이어(l1)를 정의합니다.
        self.l1 = nn.Linear(n_input, n_output)

    # 예측 함수 정의: 데이터의 순전파 흐름을 정의합니다.
    def forward(self, x):
        x1 = self.l1(x) # __init__에서 정의한 선형 레이어에 입력 x를 통과시킵니다.
        return x1

In [ ]:
# 코드 5-13 커스텀 클래스를 활용한 예측

# 더미 입력 데이터 생성 (100개의 샘플, 1개의 특성)
inputs = torch.ones(100, 1)
# print(inputs)
# 인스턴스 생성 (1 입력, 1 출력 선형 모델)
n_input = 1
n_output = 1
net = Net(n_input, n_output) # Net 클래스의 인스턴스를 생성합니다.

# 예측 수행
# net 인스턴스를 함수처럼 호출하면, 내부적으로 forward 메서드가 실행됩니다.
outputs = net(inputs)

print(outputs.shape)
# 처음 5개 예측값 출력
print(outputs[:5])

In [ ]:
# 코드 5-14 손실 함수의 정의(초기화 처리에 포함)

# 손실 함수 정의: 평균 제곱 오차(Mean Square Error)
criterion = nn.MSELoss()

# 코드 5-15 손실 함수의 호출(반복 처리에 포함)

# 예측값(outputs)은 위에서 생성한 값을 사용하고, 정답(labels) 텐서를 임의로 생성합니다.
labels1 = torch.zeros(100, 1) # 정답이 모두 0이라고 가정
# print(labels1)
# 손실 계산
# "딥러닝을 위한 수학"의 결과와 일치시키기 위해 2로 나눈 값을 손실로 함
loss = criterion(outputs, labels1) / 2.0

# 경사 계산
loss.backward()

print(f"계산된 손실(loss): {loss.item()}")
# l1 레이어의 가중치에 대한 경사(gradient) 값 확인
print(f"l1.weight.grad: {net.l1.weight.grad}")

In [ ]:
# 코드 5-16 손실(loss)의 계산 방법

# 1. 예측 계산 (첫 번째 함수)
outputs = net(inputs)
# print(outputs)
# 2. 손실 계산 (두 번째 함수)
# "딥러닝을 위한 수학"의 결과와 일치시키기 위해 2로 나눈 값을 손실로 함
loss = criterion(outputs, labels1) / 2.0

print(f"합성 함수 결과(loss): {loss.item()}")


## 5.6 데이터 준비
UCI 공개 데이터셋 중에서 회귀 문제에 자주 사용되는 '보스턴 데이터셋'을 사용한다.

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

원본 데이터셋은, 17개 항목의 입력값으로부터 부동산 가격을 예측하기 위함이지만, 가장 단순한 '단일 회귀 모델'(1입력)을 만든다면 이 항목중에서 ``RM``이라는 항목만을 추출한다.


In [ ]:
# 학습용 데이터셋 준비

# '보스턴 데이터셋'은 현재 사이킷런 라이브러리에서 가져올 수 있지만,
# 사이킷런에서 앞으로 이 데이터를 사용할 수 없기 때문에 웹 url에서 직접 수집

data_url = "http://lib.stat.cmu.edu/datasets/boston"
# 원본 데이터
raw_df = pd.read_csv(data_url, sep="\s+",
                     skiprows=22, header=None)

# print(raw_df)
x_org = np.hstack([raw_df.values[::2, :],
                   raw_df.values[1::2, :2]])
# print(x_org)
yt = raw_df.values[1::2, 2]
# print(yt)
feature_names = np.array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX',
                          'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO','B', 'LSTAT'])

# 결과 확인
print('원본 데이터', x_org.shape, yt.shape)
print('항목명: ', feature_names)

In [ ]:
# 코드 5-18: 입력 데이터 추출

# 데이터 추출(RM 항목)
x = x_org[:, feature_names == 'RM'] # RM 특성만 선택하여 x에 저장
# print(x)
print('추출 후', x.shape)
print(x[:5, :])

# 정답 데이터 표시
print('정답 데이터')
print(yt[:5])

In [ ]:
# 코드 5-19: 산포도 출력

# 산포도 출력
plt.scatter(x, yt, s=10, c='b') # x축은 방 개수, y축은 가격
plt.xlabel('방 개수')
plt.ylabel('가격')
plt.title('방 개수와 가격의 산포도')
plt.grid(True)
plt.show()

## 5.7 모델 정의

In [ ]:
x.shape[1]

In [ ]:
# 코드 5-20: 변수 정의

# 입력 차원수 (특성이 1개: RM)
n_input = x.shape[1]
# 출력 차원수 (예측할 값 1개: Price)
n_output = 1

print(f'입력 차원수: {n_input} 출력 차원수: {n_output}')

In [ ]:
# 코드 5-21: 머신러닝 모델(예측 모델)의 클래스 정의

# 머신러닝 모델(예측 모델)의 클래스 정의
class Net(nn.Module):
    def __init__(self, n_input, n_output): # 모델의 구조를 정의
        # 부모 클래스 nn.Module의 초기화 호출
        super().__init__()
        # 출력층 정의: 입력 차원에서 출력 차원으로 가는 선형 변환
        self.l1 = nn.Linear(n_input, n_output)

        # 초깃값을 모두 1로 설정
        # "딥러닝을 위한 수학"과 조건을 맞추기 위함
        nn.init.constant_(self.l1.weight, 1.0)
        nn.init.constant_(self.l1.bias, 1.0)

    # 예측 함수 정의
    def forward(self, x): # 데이터가 모델을 통과하는 흐름을 정의
        x1 = self.l1(x) # 선형 회귀 계산
        return x1

In [ ]:
# 코드 5-22: 인스턴스 생성

# 인스턴스 생성
# 1입력 1출력 선형 모델
net = Net(n_input, n_output)

In [ ]:
# 코드 5-23: 모델 안의 파라미터 목록 표시

# 모델 안의 파라미터를 확인
# 모델 안의 변수를 가져오기 위해 named_parameters 함수를 사용함
# 첫번째 요소는 변수명, 두번째 요소가 변숫값
for parameter in net.named_parameters():
    print(f'변수명: {parameter[0]}')
    print(f'변숫값: {parameter[1].data}')

In [ ]:
# 코드 5-24: parameters 함수의 사용 예

# 파라미터의 리스트를 가져오기 위해 parameters 함수를 사용
for parameter in net.parameters():
    print(parameter)

### 모델 확인

In [ ]:
# 코드 5-25: 모델의 개요 표시 1

print(net)

In [ ]:
# 코드 5-26: 모델의 개요 표시 2

# torchinfo 라이브러리를 사용하면 더 상세한 정보 확인 가능
from torchinfo import summary

summary(net, (1,))

### 손실 함수와 최적화 함수

In [ ]:
# 코드 5-27: 손실 함수와 최적화 함수 정의

# 손실 함수: 평균 제곱 오차(Mean Squared Error)
criterion = nn.MSELoss()

# 학습률
lr = 0.01

# 최적화 함수: 경사 하강법(Stochastic Gradient Descent)
# net.parameters() : 파라미터 조정(파라미터: 학습대상, w,b)
optimizer = optim.SGD(net.parameters(), lr=lr)

## 5.8 경사 하강법

In [ ]:
# 코드 5-28: 입력값 x와 정답 yt의 텐서 변수화

# 입력값 x와 정답 yt의 텐서 변수화
inputs = torch.tensor(x).float()
labels = torch.tensor(yt).float()

# 차원 수 확인
print(inputs.shape)
print(labels.shape)

In [ ]:
# 코드 5-29: 정답 labels를 (N, 1) 차원 행렬로 변형

# 손실 계산을 위해 labels를 (N, 1) 차원의 행렬로 변환
labels1 = labels.view((-1, 1))

# 차원 수 확인
print(labels1.shape)

In [ ]:
# 코드 5-30: 예측 계산

# 예측 계산
outputs = net(inputs)
# print(outputs) # 모델의 예측값 >> y_hat(y_pred)

In [ ]:
# 코드 5-31: 손실 값 계산

# 손실 값 계산
loss = criterion(outputs, labels1)
# (예측값, 실제값) 비교 >> 손실 계산

# 손실 값 가져오기
print(f'{loss.item():.5f}')

In [ ]:
# 코드 5-32: 계산 그래프 시각화

# 손실을 그래프로 나타내기
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

In [ ]:
# 코드 5-33: 경사 계산과 결과 확인

# 경사 계산
loss.backward()

# 경사 계산 결과를 확인
print(net.l1.weight.grad)
print(net.l1.bias.grad)

In [ ]:
# 코드 5-34: 파라미터 수정과 결과 확인

# 파라미터 수정
optimizer.step()

# 수정된 파라미터 확인
print(net.l1.weight)
print(net.l1.bias)

In [ ]:
# 코드 5-35: 경삿값 초기화

# 경삿값 초기화
optimizer.zero_grad()

# 경삿값이 0으로 초기화되었는지 확인
print(net.l1.weight.grad)
print(net.l1.bias.grad)

### 반복 계산

In [ ]:
# 코드 5-36: 초기화 처리

# 학습률
lr = 0.01

# 인스턴스 생성 (학습 시작 전 파라미터 값 초기화)
net = Net(n_input, n_output)

# 손실 함수: 평균 제곱 오차
criterion = nn.MSELoss()

# 최적화 함수: 경사 하강법
optimizer = optim.SGD(net.parameters(), lr=lr)

# 반복 횟수
num_epochs = 5000

# 평가 결과 기록 (손실 값만 기록)
history = np.zeros((0, 2))

# 코드 5-37: 반복 처리

# 반복 계산 메인 루프
for epoch in range(num_epochs):
    # ① 경삿값 초기화 (가장 먼저 수행!)
    optimizer.zero_grad()

    # ② 예측 계산
    outputs = net(inputs)

    # ③ 손실 계산
    # "딥러닝을 위한 수학"에 나온 결과와 맞추기 위해 2로 나눈 값을 손실로 정의
    loss = criterion(outputs, labels1) / 2.0

    # ④ 경사 계산
    loss.backward()

    # ⑤ 파라미터 수정
    optimizer.step()

    # 1000회마다 경과를 기록하고 출력
    if (epoch % 1000 == 0):
        history = np.vstack((history, np.array([epoch, loss.item()])))
        print(f'Epoch {epoch} loss: {loss.item():.5f}')

## 5.9 결과 확인

In [ ]:
# 코드 5-38 초기 손실값과 최종 손실값

# 초기 손실값과 최종 손실값
print(f'초기 손실값: {history[0,1]:.5f}')
print(f'최종 손실값: {history[-1,1]:.5f}')

In [ ]:
# 코드 5-39 손실 값을 통한 학습 곡선 출력

# 학습 곡선 출력(손실)
# 가장 처음 요소는 제외
import matplotlib.pyplot as plt

plt.plot(history[1:,0], history[1:,1], 'b')
plt.xlabel('반복 횟수')
plt.ylabel('손실')
plt.title('학습 곡선(손실)')
plt.show()

In [ ]:
np.array((x.min(), x.max())).reshape(-1,1)

In [ ]:
# 회귀 직선 산출

# x의 최솟값, 최댓값
xse = np.array((x.min(), x.max())).reshape(-1,1)
Xse = torch.tensor(xse).float()

with torch.no_grad():
  Yse = net(Xse)

print(Yse.numpy())

In [ ]:
# 코드 5-40 학습 데이터의 산포도와 회귀 직선 출력

# 산포도와 회귀 직선 출력
plt.scatter(x, yt, s=10, c='b')
plt.xlabel('방 개수')
plt.ylabel('가격')
plt.plot(Xse.data, Yse.data, c='k')
plt.title('산포도와 회귀 직선')
plt.show()

## 5.10 중회귀 모델로 확장

In [ ]:
# 코드 5-41 입력 변수 x2의 작성

# 열(LSTAT: 저소득자 비율) 추가
x_add = x_org[:, feature_names == 'LSTAT']
x2 = np.hstack((x, x_add))

# shape 표시
print(x2.shape)

# 입력 데이터 x2 표시
print(x2[:5,:])

In [ ]:
# 코드 5-42 입력 차원수를 2로 하는 모델 인스턴스 재생성
n_input = x2.shape[1]
print(n_input)

# 모델 인스턴스 생성
net = Net(n_input, n_output)

In [ ]:
# 코드 5-43 named_parameters 함수의 호출

# 모델 안의 파라미터 확인
# predict.weight가 2차원으로 바뀜
for parameter in net.named_parameters():
    print(f'변수명: {parameter[0]}')
    print(f'변숫값: {parameter[1].data}')

In [ ]:
# 코드 5-44 모델의 개요 표시 1
print(net)

In [ ]:
# 코드 5-45 모델의 개요 표시 2
summary(net, (2,))

In [ ]:
# 코드 5-46 입력 변수 inputs 재정의

# 입력 변수 x2를 텐서로 변환
# labels, labels1은 이전과 같음
inputs = torch.tensor(x2).float()

### 반복 계산

In [ ]:
# 초기화 처리

# 학습률
lr = 0.01

# 인스턴스 생성(파라미터 값 초기화)
net = Net(n_input, n_output)

# 손실 함수：평균 제곱 오차
criterion = nn.MSELoss()

# 최적화 함수 : 경사 하강법
optimizer = optim.SGD(net.parameters(), lr=lr)

# 반복 횟수
num_epochs = 50000

# 평가 결과 기록(손실 값만 기록)
history = np.zeros((0,2))

In [ ]:
# 반복 계산 메인 루프

for epoch in range(num_epochs):

    # 경삿값 초기화
    optimizer.zero_grad()

    # 예측 계산
    outputs = net(inputs)

    # 오차 계산
    # "딥러닝을 위한 수학"에 나온 결과와 맞추기 위해 2로 나눈 값을 손실로 정의
    loss = criterion(outputs, labels1) / 2.0

    # 경사 계산
    loss.backward()

    # 파라미터 수정
    optimizer.step()

    # 100회 마다 도중 경과를 기록
    if ( epoch % 100 == 0):
        history = np.vstack((history, np.array([epoch, loss.item()])))
        print(f'Epoch {epoch} loss: {loss.item():.5f}')


# loss:nan >> not a number (값이 없음)>> 발산

## 5.11 학습률의 변경

In [ ]:
# 코드 5-48 새로운 초기 설정값

# 반복 횟수
# num_epochs = 50000에서 2000으로 수정
num_epochs = 2000

# 학습률
# lr = 0.01에서 0.001로 수정
lr = 0.001

# 모델 인스턴스 생성
net = Net(n_input, n_output)

# 최적화 함수: 경사 하강법
optimizer = optim.SGD(net.parameters(), lr=lr)

# 손실 함수 : 평균 제곱 오차
criterion = nn.MSELoss()

In [ ]:
# 반복 계산 메인 루프

# 평가 결과 기록(손실 값만 기록)
history = np.zeros((0,2))

for epoch in range(num_epochs):

    # 경삿값 초기화
    optimizer.zero_grad()

    # 예측 계산
    outputs = net(inputs)

    # 오차 계산
    loss = criterion(outputs, labels1) / 2.0

    # 경사 계산
    loss.backward()

    # 파라미터 수정
    optimizer.step()

    # 500회 마다 도중 경과를 기록
    if ( epoch % 500 == 0):
        history = np.vstack((history, np.array([epoch, loss.item()])))
        print(f'Epoch {epoch} loss: {loss.item():.5f}')

In [ ]:
# 코드 5-49 실행 결과 요약

# 초기 손실값, 최종 손실값
print(f'초기 손실값: {history[0,1]:.5f}')
print(f'최종 손실값: {history[-1,1]:.5f}')

In [ ]:
# 코드 5-50 학습 곡선 출력

# 학습 곡선 출력(손실)
plt.plot(history[1:,0], history[1:,1], 'b')
plt.xlabel('반복 횟수')
plt.ylabel('손실')
plt.title('학습 곡선(손실)')
plt.show()